In [1]:
import gymnasium as gym
import socnavgym
import torch
import torch.nn as nn
import torch.nn.functional as F
from socnavgym.wrappers import DiscreteActions
from socnavgym.wrappers import ExpertObservations
from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from math import sqrt
import argparse
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.results_plotter import ts2xy, plot_results
from stable_baselines3.common.utils import safe_mean
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import imageio
from socnavgym.envs.utils.continuous_obs_render import ContinuousObsRenderer


/home/adonis320/miniconda3/envs/socnavgym/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
env = gym.make("SocNavGym-v1", config="./configs/4_humans.yaml") 
from stable_baselines3 import DQN
env = DiscreteActions(env)
env = ExpertObservations(env)

model = DQN.load("./4_humans/rl_model_300000_steps.zip", env=env)

/home/adonis320/miniconda3/envs/socnavgym/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:54: UserWarning: WARN: A Box action space maximum and minimum values are equal.
  logger.warn("A Box action space maximum and minimum values are equal.")


In [3]:
import cv2
obs = env.reset()
obs, info = env.reset()

# IMPORTANT: use the base env for rendering utilities
base_env = env.unwrapped

renderer = ContinuousObsRenderer(
    env=base_env,
    extent_m=6.0,           # local view range in meters
    show_world=True,        # world || local
    draw_rings=True,
    draw_axes=True,
    draw_robot_glyph=True,
)

while True:
    renderer.render(obs, show=True, delay_ms=60)

    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break

    if terminated or truncated:
        obs, info = env.reset()

env.close()
cv2.destroyAllWindows()

QFontDatabase: Cannot find font directory /home/adonis320/miniconda3/envs/socnavgym/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/adonis320/miniconda3/envs/socnavgym/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/adonis320/miniconda3/envs/socnavgym/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/adonis320/miniconda3/envs/socnavgym/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconf

KeyboardInterrupt: 

In [ ]:
obs, _ = env.reset()
frames = []

for i in range(600):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    #next_state_key = self.get_state_key(obs)
#
    #        # Accumulate reward
    #        episode_reward += reward
#
    #        # Update SR and branch rewards
    #        self.update_sr(state_key, action, next_state_key, done or truncated, upd_social=upd_social)
    #        self.update_reward(state_key, action, reward, upd_social=upd_social)
    frame = env.render()
    frames.append(frame)
    if terminated or truncated:
        break

env.close()
#imageio.mimsave("./dqn_2_humans.gif", frames, fps=20)